In [112]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import talib as ta
from category_encoders import TargetEncoder
from scipy.stats import mode

%matplotlib inline

In [113]:
file_path = 'csv_files/bitcoin_data_bitcoininfocharts.csv'

# Read the CSV file
df_bitinfo = pd.read_csv(file_path)
df_bitinfo['Date'] = pd.to_datetime(df_bitinfo['Date'])  # Convert the 'Date' column to datetime objects
df_bitinfo.set_index('Date', inplace=True)  # Set the 'Date' column as the DataFrame index

df_bitinfo.head()

,btc_price,btc_trans_blockchain,avg_block_size,unique_sentbyaddress,avg_mining_diff,avg_hashrate,sent_in_USD,avg_trans_fee,median_trans_fee,avg_block_confirm_time_min,avg_trans_value_usd,median_trans_value_usd,unique_tofromaddress
Date,,,,,,,,,,,,,
2009-03-01,NaN,NaN,204.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [114]:
file_path = 'csv_files/btc_usd_sentiment.csv'

# Read the CSV file
df_senti = pd.read_csv(file_path)
df_senti['Date'] = pd.to_datetime(df_senti['Date'])  # Convert the 'Date' column to datetime objects
df_senti['Date'] = df_senti['Date'].dt.date
df_senti.set_index('Date', inplace=True)  # Set the 'Date' column as the DataFrame index

df_senti.head()

,Open,High,Low,Close,Adj Close,Volume,Sentiment
Date,,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800,1
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200,1
2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700,-1
2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600,-1
2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100,0


In [115]:
df              = df_senti.join(df_bitinfo, how='left', sort=True)
df.columns      = df.columns.str.lower()
drop_columns    = ['btc_price', 'avg_trans_fee', 'avg_trans_value_usd', 'avg_mining_diff', 'unique_tofromaddress'] # Filtering not required indicators
df              = df.drop(columns=drop_columns)
df

,open,high,low,close,adj close,volume,sentiment,btc_trans_blockchain,avg_block_size,unique_sentbyaddress,avg_hashrate,sent_in_usd,median_trans_fee,avg_block_confirm_time_min,median_trans_value_usd
Date,,,,,,,,,,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800,1,80119.0,240563.0,133776.0,2.730000e+17,1.694316e+08,0.0456,7.742,110.753
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200,1,77185.0,236621.0,134311.0,2.720000e+17,2.314617e+08,0.0439,8.045,131.235
2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700,-1,69266.0,221271.0,117316.0,2.590000e+17,2.307807e+08,0.0406,8.421,130.667
2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600,-1,63306.0,208320.0,107878.0,2.500000e+17,2.404431e+08,0.0410,8.182,94.343
2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100,0,59636.0,217914.0,102979.0,2.600000e+17,2.728374e+08,0.0401,8.780,80.706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-02,69705.023438,69708.382813,64586.593750,65446.972656,65446.972656,50705240709,1,336056.0,803328.0,393821.0,5.820000e+20,6.537140e+09,2.8750,9.290,120.205
2024-04-03,65446.671875,66914.320313,64559.898438,65980.812500,65980.812500,34488018367,1,361129.0,740862.0,440206.0,5.310000e+20,2.952809e+10,7.5340,11.163,467.308
2024-04-04,65975.695313,69291.257813,65113.796875,68508.843750,68508.843750,34439527442,1,320469.0,756407.0,446917.0,5.750000e+20,2.430922e+10,1.9050,10.435,356.076


In [116]:
## General Technical Indicators

# Calculate Simple Moving Average (SMA) and Exponential Moving Average (EMA)
df['sma_20'] = df['close'].rolling(window=20).mean() 
df['ema_20'] = df['close'].ewm(span=20, adjust=False).mean()

# Calculate MACD (taken from https://www.investopedia.com/ask/answers/122414/what-moving-average-convergence-divergence-macd-formula-and-how-it-calculated.asp)
df['ema_12']      = df['close'].ewm(span=12, adjust=False).mean()
df['ema_26']      = df['close'].ewm(span=26, adjust=False).mean()
df['macd']        = df['ema_12'] - df['ema_26']
df['signal_line'] = df['macd'].ewm(span=9, adjust=False).mean()

# Calculate RSI (taken from https://www.investopedia.com/terms/r/rsi.asp)
delta = df['close'].diff()
up = delta.clip(lower=0)
down = -1 * delta.clip(upper=0)
ema_up = up.ewm(com=13, adjust=False).mean()
ema_down = down.ewm(com=13, adjust=False).mean()
rs = ema_up / ema_down
df['rsi'] = 100 - (100 / (1 + rs))

# Calculate Stochastic Oscillator (%K and %D) (taken from https://www.investopedia.com/terms/s/stochasticoscillator.asp)
low_14      = df['low'].rolling(window=14).min()
high_14     = df['high'].rolling(window=14).max()
df['%k']    = 100 * ((df['close'] - low_14) / (high_14 - low_14)) # fast oscillator
df['%d']    = df['%k'].rolling(window=3).mean() # slow oscillator

# Calculate Bollinger Bands
df['middle_band']   = df['sma_20']
df['upper_band']    = df['middle_band'] + 2*df['close'].rolling(window=20).std()
df['lower_band']    = df['middle_band'] - 2*df['close'].rolling(window=20).std()

# Display the head of the DataFrame with new columns
df

,open,high,low,close,adj close,volume,sentiment,btc_trans_blockchain,avg_block_size,unique_sentbyaddress,...,ema_12,ema_26,macd,signal_line,rsi,%k,%d,middle_band,upper_band,lower_band
Date,,,,,,,,,,,,,,,,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800,1,80119.0,240563.0,133776.0,...,457.334015,457.334015,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200,1,77185.0,236621.0,134311.0,...,452.273398,454.897421,-2.624024,-0.524805,0.000000,NaN,NaN,NaN,NaN,NaN
2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700,-1,69266.0,221271.0,117316.0,...,443.430720,450.445464,-7.014744,-1.822793,0.000000,NaN,NaN,NaN,NaN,NaN
2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600,-1,63306.0,208320.0,107878.0,...,438.118915,447.368318,-9.249402,-3.308115,3.215775,NaN,NaN,NaN,NaN,NaN
2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100,0,59636.0,217914.0,102979.0,...,432.073084,443.772221,-11.699137,-4.986319,3.138104,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-02,69705.023438,69708.382813,64586.593750,65446.972656,65446.972656,50705240709,1,336056.0,803328.0,393821.0,...,68545.423157,66954.927305,1590.495851,2076.985378,47.286223,42.483782,73.442092,67930.776172,73408.050919,62453.501425
2024-04-03,65446.671875,66914.320313,64559.898438,65980.812500,65980.812500,34488018367,1,361129.0,740862.0,440206.0,...,68150.867671,66882.770653,1268.097018,1915.207706,48.395257,38.682448,54.205724,67659.987110,72948.037016,62371.937203
2024-04-04,65975.695313,69291.257813,65113.796875,68508.843750,68508.843750,34439527442,1,320469.0,756407.0,446917.0,...,68205.940914,67003.220512,1202.720402,1772.710245,53.395639,65.655836,48.940688,67615.240625,72856.096582,62374.384668


In [117]:
df.replace([np.inf, -np.inf], np.nan)
columns_to_fill = ['btc_trans_blockchain', 'avg_block_size', 'avg_hashrate', 'unique_sentbyaddress']
df[columns_to_fill] = df[columns_to_fill].fillna(df[columns_to_fill].mean())
columns_to_fill2 = ['sent_in_usd', 'median_trans_fee', 'avg_block_confirm_time_min', 'median_trans_value_usd']
df[columns_to_fill2] = df[columns_to_fill2].fillna(method='ffill')
df

,open,high,low,close,adj close,volume,sentiment,btc_trans_blockchain,avg_block_size,unique_sentbyaddress,...,ema_12,ema_26,macd,signal_line,rsi,%k,%d,middle_band,upper_band,lower_band
Date,,,,,,,,,,,,,,,,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800,1,80119.000000,240563.000000,133776.000000,...,457.334015,457.334015,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200,1,77185.000000,236621.000000,134311.000000,...,452.273398,454.897421,-2.624024,-0.524805,0.000000,NaN,NaN,NaN,NaN,NaN
2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700,-1,69266.000000,221271.000000,117316.000000,...,443.430720,450.445464,-7.014744,-1.822793,0.000000,NaN,NaN,NaN,NaN,NaN
2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600,-1,63306.000000,208320.000000,107878.000000,...,438.118915,447.368318,-9.249402,-3.308115,3.215775,NaN,NaN,NaN,NaN,NaN
2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100,0,59636.000000,217914.000000,102979.000000,...,432.073084,443.772221,-11.699137,-4.986319,3.138104,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-02,69705.023438,69708.382813,64586.593750,65446.972656,65446.972656,50705240709,1,336056.000000,803328.000000,393821.000000,...,68545.423157,66954.927305,1590.495851,2076.985378,47.286223,42.483782,73.442092,67930.776172,73408.050919,62453.501425
2024-04-03,65446.671875,66914.320313,64559.898438,65980.812500,65980.812500,34488018367,1,361129.000000,740862.000000,440206.000000,...,68150.867671,66882.770653,1268.097018,1915.207706,48.395257,38.682448,54.205724,67659.987110,72948.037016,62371.937203
2024-04-04,65975.695313,69291.257813,65113.796875,68508.843750,68508.843750,34439527442,1,320469.000000,756407.000000,446917.000000,...,68205.940914,67003.220512,1202.720402,1772.710245,53.395639,65.655836,48.940688,67615.240625,72856.096582,62374.384668


In [120]:
# Remove rows of data with NaNs on inspection
df = df.iloc[50:-5]
df

,open,high,low,close,adj close,volume,sentiment,btc_trans_blockchain,avg_block_size,unique_sentbyaddress,...,ema_12,ema_26,macd,signal_line,rsi,%k,%d,middle_band,upper_band,lower_band
Date,,,,,,,,,,,,,,,,,,,,,
2014-12-26,319.152008,331.424011,316.627014,327.924011,327.924011,16410500,1,76046.0,274839.0,126447.0,...,329.827416,340.746992,-10.919576,-10.878494,43.046049,48.268293,36.803432,337.289447,371.566174,303.012721
2014-12-27,327.583008,328.911011,312.630005,315.863007,315.863007,15185200,-1,76662.0,290886.0,130695.0,...,327.679046,338.903734,-11.224688,-10.947733,38.350869,23.696130,34.022637,334.327847,364.884636,303.771059
2014-12-28,316.160004,320.028015,311.078003,317.239014,317.239014,11676600,1,75546.0,279938.0,144477.0,...,326.072887,337.298940,-11.226053,-11.003397,39.166110,27.335429,33.099950,332.094348,360.625754,303.562942
2014-12-29,317.700989,320.266998,312.307007,312.670013,312.670013,12302500,-1,83283.0,284084.0,135586.0,...,324.010906,335.474575,-11.463669,-11.095451,37.397674,20.270534,23.767364,330.116899,358.254878,301.978921
2014-12-30,312.718994,314.808990,309.372986,310.737000,310.737000,12528300,0,84894.0,301133.0,144680.0,...,321.968767,333.642162,-11.673395,-11.211040,36.643836,20.293267,22.633076,328.335500,356.652893,300.018106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-23,63802.722656,65976.398438,63038.492188,64062.203125,64062.203125,24738964812,-1,326454.0,735606.0,428273.0,...,66051.851489,64272.686197,1779.165292,3083.295524,49.978567,25.145621,28.096532,67644.243360,73697.443547,61591.043172
2024-03-24,64070.753906,67622.757813,63825.851563,67234.171875,67234.171875,27206630673,1,331357.0,752623.0,394673.0,...,66233.746933,64492.055506,1741.691427,2814.974705,55.088041,49.654189,32.585130,67589.431250,73636.319888,61542.542612
2024-03-25,67234.093750,71162.593750,66414.835938,69958.812500,69958.812500,42700139523,1,316932.0,693220.0,404678.0,...,66806.833943,64897.000469,1909.833475,2633.946459,58.965394,70.706427,48.502079,67897.311914,73756.188239,62038.435590


In [121]:
df.to_csv('csv_files/bitcoin_techindicators_filtered.csv', index=True, encoding='utf-8')

In [122]:
df.columns.to_list()


['open',
 'high',
 'low',
 'close',
 'adj close',
 'volume',
 'sentiment',
 'btc_trans_blockchain',
 'avg_block_size',
 'unique_sentbyaddress',
 'avg_hashrate',
 'sent_in_usd',
 'median_trans_fee',
 'avg_block_confirm_time_min',
 'median_trans_value_usd',
 'sma_20',
 'ema_20',
 'ema_12',
 'ema_26',
 'macd',
 'signal_line',
 'rsi',
 '%k',
 '%d',
 'middle_band',
 'upper_band',
 'lower_band']

In [123]:
volume_like_indicators = ['btc_trans_blockchain', 'avg_block_size', 'avg_hashrate', 'sent_in_usd']
price_like_indicators = ['median_trans_fee', 'median_trans_value_usd', 'sma_20', 'ema_20', 'ema_12', 'ema_26', 'macd', 'signal_line', 'rsi', '%k', '%d', 'middle_band', 'upper_band', 'lower_band']

In [124]:
#Loop A          
#volume like
for i in volume_like_indicators:   
    df[i] = np.log1p(df[i])
    mean_short = df[i].rolling(window=10).mean()
    mean_long = df[i].rolling(window=200).mean()
    oscillator = mean_short-mean_long
    df[i] = oscillator


#make the data stationary by creating an oscillator scaled by the inter quartile range 
for i in price_like_indicators:   
    if ((df[i].dtype == 'int64') or (df[i].dtype == 'float64')):
        df[i] = np.log1p(df[i])
        mean_short = df[i].ewm(span=1).mean()
        mean_long = df[i].ewm(span=10).mean() 
        oscillator = mean_short-mean_long
        quantile_25 = oscillator.rolling(100).quantile(0.25)
        quantile_75 = oscillator.rolling(100).quantile(0.75)
        iqr = quantile_75 - quantile_25
        #df[i] = oscillator/iqr 
        df[i] = oscillator #NO NEED TO SCALE IF USING RANDOM FOREST


In [133]:
df.replace([np.inf, -np.inf], np.nan)
columns = df.columns.to_list()
df[columns] = df[columns].fillna(df[columns].mean())
df = df.iloc[1:]
df

,open,high,low,close,adj close,volume,sentiment,btc_trans_blockchain,avg_block_size,unique_sentbyaddress,...,ema_12,ema_26,macd,signal_line,rsi,%k,%d,middle_band,upper_band,lower_band
Date,,,,,,,,,,,,,,,,,,,,,
2014-12-28,316.160004,320.028015,311.078003,317.239014,317.239014,11676600,1,0.038075,0.016595,144477.0,...,-0.004683,-0.004285,-0.508608,-0.681881,-0.018070,-0.103646,-0.036527,-0.006363,-0.011871,0.000262
2014-12-29,317.700989,320.266998,312.307007,312.670013,312.670013,12302500,-1,0.038075,0.016595,135586.0,...,-0.007381,-0.006499,-0.508608,-0.681881,-0.042309,-0.261805,-0.238912,-0.008259,-0.012371,-0.003321
2014-12-30,312.718994,314.808990,309.372986,310.737000,310.737000,12528300,0,0.038075,0.016595,144680.0,...,-0.009756,-0.008527,-0.508608,-0.681881,-0.044299,-0.185886,-0.203749,-0.009734,-0.012006,-0.006996
2014-12-31,310.914001,320.192993,310.210999,320.192993,320.192993,13942900,-1,0.038075,0.016595,122759.0,...,-0.007848,-0.008519,-0.508608,-0.681881,0.078077,0.505951,0.052866,-0.010620,-0.015802,-0.004458
2015-01-01,320.434998,320.434998,314.002991,314.248993,314.248993,8036550,-1,0.038075,0.016595,93422.0,...,-0.008657,-0.009573,-0.508608,-0.681881,0.012601,-0.114383,0.080269,-0.012508,-0.021293,-0.002163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-23,63802.722656,65976.398438,63038.492188,64062.203125,64062.203125,24738964812,-1,-0.140453,-0.003747,428273.0,...,-0.006374,0.015545,-0.508481,-0.239341,-0.111988,-0.318888,-0.461249,0.025145,-0.003951,0.063527
2024-03-24,64070.753906,67622.757813,63825.851563,67234.171875,67234.171875,27206630673,1,-0.139135,-0.006067,394673.0,...,-0.002965,0.015506,-0.433433,-0.270286,-0.013476,0.280188,-0.260005,0.019910,-0.003912,0.051332
2024-03-25,67234.093750,71162.593750,66414.835938,69958.812500,69958.812500,42700139523,1,-0.133062,-0.016933,404678.0,...,0.004623,0.017808,-0.279264,-0.275503,0.043666,0.513611,0.104667,0.020008,-0.001870,0.048565


In [134]:
df.to_csv('csv_files/bitcoin_techindicators_filtered_normalized.csv', index=True, encoding='utf-8')